# Exploración y descarga de datos de RNA-Seq

## Pregunta

> ¿Cómo podemos acceder, explorar y descargar datos de RNA-Seq asociados a un BioProject específico (PRJNAXXX), para su posterior análisis transcriptómico, incluyendo el genoma de referencia necesario para el alineamiento de las lecturas?

## Objetivos

**General:**

> Entender el proceso de exploración y descarga de datos de RNA-Seq y del genoma de referencia desde NCBI, siguiendo la relación BioProject → GEO → GSM → SRR → Genoma, utilizando Biopython y SRA Toolkit.

**Específicos:**

> 1. Comprender la estructura jerárquica de NCBI: BioProject, BioSample, GEO Series (GSE), GEO Samples (GSM) y SRA Runs (SRR).  
> 2. Obtener información de un BioProject específico usando Biopython (`esearch`, `esummary`, `elink`).  
> 3. Identificar las muestras (GSM) y sus archivos de secuenciación (SRR) asociados al proyecto.  
> 4. Descargar los archivos SRR y convertirlos a formato FASTQ utilizando SRA Toolkit.  
> 5. Combinar archivos de lecturas *forward* y *reverse* cuando corresponda, generando archivos finales representativos de cada muestra.  
> 6. Consultar y descargar el **genoma de referencia** del organismo correspondiente, incluyendo secuencias y archivos de anotación (GFF), para el posterior mapeo y análisis de expresión diferencial.


## 1. Problemática: Pasilla en *Drosophila melanogaster*

Como vimos en clases anteriores, el proyecto que abordaremos está relacionado con la expresión del gen **Pasilla**. El experimento evaluó cómo la inhibición de este gen, ortólogo en *Drosophila melanogaster* de los genes **NOVA1** y **NOVA2** en mamíferos, afecta el **transcriptoma**. Para ello, se analizaron datos de **RNA-Seq** depositados en [**NCBI-GEO**](https://www.ncbi.nlm.nih.gov/geo/) (accesos **GSM461176** a **GSM461182**) y asociados al **BioProject [PRJNA168994](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA168994)**, según el estudio publicado por **[Brooks et al.](https://genome.cshlp.org/content/21/2/193)** en *Genome Research*. En esta clase, nos enfocaremos en la descarga y consulta de los datos asociados al proyecto [**PRJNA168994**](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA168994), prestando especial atención a las muestras **GSM461176** a **GSM461182**, y revisaremos de manera general el uso de [**Biopython**](https://biopython.org/) y [**SRA Toolkit**](https://github.com/ncbi/sra-tools) para la obtención de los datos.


## 2. Conceptos clave: BioProject, BioSample, SRA y GEO

De manera general, un **BioProject (PRJNAxxxxxx)** describe el proyecto de investigación en su conjunto (por ejemplo, un estudio de RNA-Seq) y se vincula a una o varias **BioSamples (SAMNxxxxxx)**, que documentan las características biológicas de cada muestra. A partir de estas BioSamples se generan los datos crudos de secuenciación, almacenados en el [**Sequence Read Archive (SRA, SRRxxxxxx)**](https://www.ncbi.nlm.nih.gov/sra).

En algunos casos, el proyecto también deposita información en **Gene Expression Omnibus (GEO)**:  

- Cada **GSE** corresponde a un experimento completo.  
- Cada **GSM** corresponde a una muestra individual, asociada a sus archivos de secuenciación.  

En resumen, la ruta **BioProject → BioSample → SRA** está siempre presente, mientras que la rama hacia **GEO** es opcional y depende del tipo de estudio.


## 3. Exploración de datos en NCBI con Biopython

Antes de descargar datos, podemos obtener información general sobre un **BioProject** (por ejemplo, [**PRJNA168994**](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA168994)) utilizando la biblioteca [**Biopython**](https://biopython.org/) y sus funciones de `Bio.Entrez`. 

### Resumen de funciones de `Bio.Entrez`

- **`esearch: `** Buscar IDs en una base de datos.  
- **`efetch: `** Descargar información completa (texto, XML, secuencias, metadatos).  
- **`esummary: `** Obtener un resumen de un ID (título, organismo, etc.).  
- **`elink: `** Encontrar vínculos entre bases de datos (ej. GSE → SRA).  
- **`egquery: `** Buscar un término en **todas las bases del NCBI** y ver cuántos resultados hay.  
- **`espell: `** Corregir errores ortográficos en la búsqueda.  
- **`einfo: `** Ver información de una base de datos (campos, descripción, qué contiene).  
- **`read: `** convierte la respuesta de NCBI, en un **objeto de python**(listas, diccionarios, strings, etc.).

### 3.1 Preparación: carga de bibliotecas
Primero, importaremos la biblioteca **Entrez** de Biopython, que permite conectarse con las bases de datos de NCBI y utilizar funciones como `esearch`, `efetch`, `esummary` y `elink` para explorar y descargar información sobre proyectos, muestras y datos de secuenciación.

> **Nota:** Para acceder a las bases de datos de NCBI es necesario proporcionar una **cuenta de correo electrónico** al utilizar Entrez, para que el servidor pueda identificar al usuario.

In [1]:
# Cargando el módulo Entrez de la biblioteca biopython y otras bibliotecas necesarias para esta sesión
from Bio import Entrez
import pandas as pd
from io import StringIO
import subprocess
import os

# Proporcionar correo electrónico
Entrez.email = 'hectorjl@lcg.unam.mx'



### 3.2 Obtener el UID de un BioProject

En NCBI no es posible acceder a un proyecto únicamente por su nombre, ya que el sistema utiliza un **identificador interno llamado UID (Unique ID)** para cada registro.

Por ello, el primer paso consiste en obtener este **UID** utilizando la función `esearch` de Biopython, especificando la base de datos **bioproject**. Con el UID obtenido, podremos consultar toda la información asociada al proyecto.


In [2]:
# Buscar el UID del BioProject
project_acc = "PRJNA168994"

# Usamos eserch para hacer una consulta y bajarla a una var
# Esto abre una busqueda despues hay que cerrarla
handle = Entrez.esearch(db='bioproject', term=project_acc)
search_results = Entrez.read(handle)

# Cerramos la busqueda
handle.close()

print(search_results)


{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['168994'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'PRJNA168994[All Fields]', 'Field': 'All Fields', 'Count': '1', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'PRJNA168994[All Fields]'}


La consulta **devuelve diversos parámetros** relacionados con los resultados de la búsqueda. A continuación se describen los más importantes:

| **Campo**            | **Significado** |
|-----------------------|-----------------|
| **`Count`**          | Número total de resultados encontrados para la búsqueda. |
| **`RetMax`**         | Máximo de resultados devueltos en la consulta actual (por defecto 20 si no se especifica). |
| **`RetStart`**       | Índice del primer resultado devuelto (sirve para paginar los resultados). |
| **`IdList`**         | Lista de identificadores únicos (**UIDs**) de los resultados encontrados. Estos UIDs se usan en `efetch`, `esummary`, etc. |
| **`TranslationSet`** | Conjunto de traducciones de términos de búsqueda que hace NCBI (ej. sinónimos o equivalencias). Puede estar vacío. |
| **`TranslationStack`** | Muestra paso a paso cómo NCBI interpretó tu búsqueda (ej. en qué campos buscó y cuántos resultados encontró). |
| **`QueryTranslation`** | Versión final de la consulta después de ser interpretada por NCBI (puede ser igual a tu búsqueda original o una forma ajustada). |


Ahora, extraemos únicamente el **UID** del bioproject a partir del resultado de la búsqueda:

In [4]:
# Extraer solo el UID

project_uid = search_results['IdList'][0]
print(project_uid)

168994


### 3.3 Resumen del BioProject

Con el **UID** obtenido anteriormente, podemos consultar un **resumen del BioProject** usando la función `esummary` de Biopython sobre la base de datos *bioproject*:

In [5]:
# Obtener resumen del BioProject
handle = Entrez.esummary(db = 'bioproject', id = project_uid)
project_record = Entrez.read(handle)
handle.close()
print(project_record)





{'DocumentSummarySet': DictElement({'DocumentSummary': [DictElement({'TaxId': '7227', 'Project_Id': '168994', 'Project_Acc': 'PRJNA168994', 'Project_Type': 'Primary submission', 'Project_Data_Type': 'Transcriptome or Gene expression', 'Sort_By_ProjectType': '886883', 'Sort_By_DataType': '884243', 'Sort_By_Organism': '130116', 'Project_Subtype': '', 'Project_Target_Scope': 'Multiisolate', 'Project_Target_Material': 'Transcriptome', 'Project_Target_Capture': 'Whole', 'Project_MethodType': 'Array', 'Project_Method': '', 'Project_Objectives_List': [{'Project_ObjectivesType': 'Expression', 'Project_Objectives': ''}], 'Registration_Date': '2012/06/20 00:00', 'Project_Name': 'Drosophila melanogaster', 'Project_Title': 'Drosophila melanogaster Transcriptome or Gene expression', 'Project_Description': 'RNA profiling data sets generated by the Drosophila modENCODE project.', 'Keyword': '', 'Relevance_Agricultural': '', 'Relevance_Medical': '', 'Relevance_Industrial': '', 'Relevance_Environmental

La función `esummary` devuelve un **diccionario** que resume la **información esencial de un BioProject**, incluyendo datos sobre el **organismo**, **tipo** y objetivo del proyecto, **fecha de registro**, estado de secuenciación y más. La siguiente estructura muestra los campos más relevantes obtenidos para el proyecto [**PRJNA168994**](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA168994):

#### Accediendo a campos específicos

Para acceder a algunos de los campos del BioProject, utilizamos el siguiente código.

In [ ]:
# Seleccionando la información relacionada al proyecto
proj = project_record['DocumentSummarySet']['DocumentSummary'][0]


# Mostrando todos los campos del diccionario
# print(proj.keys())



# Accediendo a algunos campos
print("\n=== Información del BioProject ===")
print(f"Accession: {proj['Project_Acc']}")
print(f"Organism: {proj['Organism_Name']}")
print(f"Description: {proj['Project_Description']}")





dict_keys(['TaxId', 'Project_Id', 'Project_Acc', 'Project_Type', 'Project_Data_Type', 'Sort_By_ProjectType', 'Sort_By_DataType', 'Sort_By_Organism', 'Project_Subtype', 'Project_Target_Scope', 'Project_Target_Material', 'Project_Target_Capture', 'Project_MethodType', 'Project_Method', 'Project_Objectives_List', 'Registration_Date', 'Project_Name', 'Project_Title', 'Project_Description', 'Keyword', 'Relevance_Agricultural', 'Relevance_Medical', 'Relevance_Industrial', 'Relevance_Environmental', 'Relevance_Evolution', 'Relevance_Model', 'Relevance_Other', 'Organism_Name', 'Organism_Strain', 'Organism_Label', 'Sequencing_Status', 'Submitter_Organization', 'Submitter_Organization_List', 'Supergroup'])

=== Información del BioProject ===
Accession: PRJNA168994
Organism: Drosophila melanogaster
Description: RNA profiling data sets generated by the Drosophila modENCODE project.


### 3.4 Relacionando BioProject con GEO (GSE/GSM)

Un BioProject puede estar vinculado a varias bases de datos, como **SRA (SRR/SRX), GEO (GSE/GSM) o BioSample (SRS)**. Con la función `elink` podemos consultar estas relaciones:

In [19]:
# Revisar enlaces desde BioProject hacia GEO (GDS), con el uid del bioproject
handle = Entrez.elink(dbfrom = 'bioproject', db = 'gds', id = project_uid)
links_bioproject = Entrez.read(handle)

# Closing
handle.close()

print(links_bioproject)

geo_ids = [link['Id'] for link in links_bioproject[0]['LinkSetDb'][0]['Link']]
print(f'Se han encontrado {len(geo_ids)} Geo Ids, primeros 5: {geo_ids[:5]}')

[{'LinkSetDb': [{'Link': [{'Id': '200040045'}, {'Id': '200040043'}, {'Id': '200040042'}, {'Id': '200040040'}, {'Id': '200040039'}, {'Id': '200040038'}, {'Id': '200040036'}, {'Id': '200040034'}, {'Id': '200040016'}, {'Id': '200040015'}, {'Id': '200037756'}, {'Id': '200037440'}, {'Id': '200037439'}, {'Id': '200037438'}, {'Id': '200037437'}, {'Id': '200037436'}, {'Id': '200037435'}, {'Id': '200037434'}, {'Id': '200037300'}, {'Id': '200037299'}, {'Id': '200037297'}, {'Id': '200037293'}, {'Id': '200037292'}, {'Id': '200037291'}, {'Id': '200025570'}, {'Id': '200025392'}, {'Id': '200025390'}, {'Id': '200025387'}, {'Id': '200025348'}, {'Id': '200025347'}, {'Id': '200024608'}, {'Id': '200024607'}, {'Id': '200024605'}, {'Id': '200024604'}, {'Id': '200024545'}, {'Id': '200024544'}, {'Id': '200024543'}, {'Id': '200024542'}, {'Id': '200024541'}, {'Id': '200024540'}, {'Id': '200024539'}, {'Id': '200024317'}, {'Id': '200024316'}, {'Id': '200024315'}, {'Id': '200024314'}, {'Id': '200024313'}, {'Id': '

> Nota: Cada BioProject genera un *LinkSet*, y "LinkSetDb" contiene los **UIDs** asociados en la base consultada. En este ejemplo exploramos un único BioProject, pero se pueden consultar **varios proyectos simultaneamente**, obteniendo un *LinkSet* por cada uno. En nuestro caso, el proyecto [**PRJNA168994**](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA168994) está vinculado **156 GEO Series (GSE)**, cada una representando un estudio completo.

### 3.5 Resumen de un GSE y sus muestras (GSM)

Para obtener detalles de un **GSE**, usamos nuevamente `esummary`

In [ ]:
# Verificando la información de GEO
# ejemplo, segundo UID




Al ejecutar `esummary` sobre un **UID** de GEO (**GSE**), obtenemos un **diccionario** con múltiples campos que describen el experimento. Entre la información más relevante se encuentran:  

- **Accession**: identificador tipo GSE  
- **title**: título del estudio  
- **summary**: descripción general del experimento  
- **organisms** estudiados  
- **Samples**: lista de muestras asociadas (**GSM**)  
- **Publicación**: fecha de publicación  
- **Plataforma tecnológica** utilizada  

Estos campos permiten comprender rápidamente el alcance y los objetivos del experimento sin necesidad de explorar los registros individuales de cada muestra (GSM).



#### Impresión de campos clave de un GSE

In [ ]:
# imprimiendo los campos
geo_record=summary[0]
print("=== GEO Accession ===")
print("Accession:", geo_record.get("Accession"))
print("Tipo:", geo_record.get("entryType"))
print("Título:", geo_record.get("title"))
print("Resumen:", geo_record.get("summary"))
print("Organismo:", geo_record.get("taxon"))
print("Número de muestras:", geo_record.get("n_samples"))
print("PubMed IDs:", geo_record.get("PubMedIds"))
print("Enlace FTP:", geo_record.get("FTPLink"))
print("Archivos suplementarios:", geo_record.get("suppFile"))
print("Proyectos: ",geo_record.get("Projects"))

# Muestras asociadas
print("\n=== Muestras ===")
for sample in geo_record.get("Samples",[]):
    print(sample['Accession'],":",sample['Title'])

### 3.6 Obtener SRR asociados a un GSM

Recordemos que para nuestro proyecto, nos enfocaremos en las muestras **GSM461176** a **GSM461182**, cada una con metadatos y enlaces a archivos de secuencias (SRR) disponibles en SRA. De manera particular trabajaremos con la muestra **GSM461177**

El primer paso es obtener el **UID** interno de NCBI de la muestra:

In [ ]:
# Obtener el UID correspondiente a la muestra
gsm_id = "GSM461177"





La respuesta incluye campos como **Count**, **IdList**, **RetMax**, etc, que ya fueron descritos anteriormente [sección 3.2](3.2 Obtener el **UID** de un BioProject). El **UID** de la muestra se encuentra en **IdList**. es importante mencionar que si no hay un **UID**, asociado el parámetro **IdList** será vacío.

In [ ]:
# Consultar si existe el identificador



# Obtener la información de la corrida como texto



# Cargar en pandas el resultado, directamente desde la cadena





## 4. Descarga de datos con SRA Toolkit

El **SRA Toolkit** ([sra-tools](https://github.com/ncbi/sra-tools)) es un conjunto de programas creados por el **NCBI** para descargar y manejar datos de secuenciación masiva almacenados en el [**SRA (Sequence Read Archive)**](https://www.ncbi.nlm.nih.gov/sra).  
Este toolkit incluye **diversas funciones** que permiten explorar, validar, convertir y descargar datos.

En este curso nos enfocaremos únicamente en las funciones **de descarga y conversión**, necesarias para obtener las secuencias en formato FASTQ para nuestro análisis. No obstante, el *toolkit* cuenta con otras utilidades que pueden consultarse en la [documentación oficial](https://github.com/ncbi/sra-tools) o ejecutando `--help` en cualquier comando.


**Funiones de descarga y conversión**

- **prefetch**: descarga los archivos SRR completos desde el servidor del SRA hacia tu disco local.  
- **fastq-dump / fasterq-dump**: extrae las secuencias en formato **FASTQ** desde los archivos SRR descargados. Permite separar lecturas pareadas (`--split-files`) y comprimir los resultados (`--gzip`).  

Con esto, podemos explorar los **SRR** asociados a cada muestra, conocer la plataforma, el tipo de librería y otros **metadatos** relevantes para análisis de RNA-Seq.

### 4.1 Ejecución de sra-tools desde python

Las herramientas de **SRA Toolkit** se ejecutan normalmente **desde la línea de comandos** en una terminal. Como no forman parte de Python ni de Biopython, para integrarlas dentro de un *script* usamos la biblioteca estándar **`subprocess`**, que permite **ejecutar programas externos** y capturar su salida. Esto facilita **automatizar la descarga y conversión** de múltiples archivos SRR en un flujo de análisis reproducible.



| Función                     | Descripción                                            | Parámetros clave                                   | Ejemplo                                                                                                      |
| --------------------------- | ------------------------------------------------------ | -------------------------------------------------- | ------------------------------------------------------------------------------------------------------------ |
| `subprocess.run()`          | Ejecuta un comando externo y espera a que termine      | `args`, `check`, `stdout`, `stderr`, `cwd`, `text` | `subprocess.run(["ls","-l"], check=True, stdout=subprocess.PIPE, text=True)`                                 |
| `subprocess.Popen()`        | Ejecuta un proceso y permite interactuar con él        | `stdin`, `stdout`, `stderr`, `text`, `cwd`         | `p = subprocess.Popen(["echo","Hola"], stdout=subprocess.PIPE, text=True); output, errors = p.communicate()` |
| `subprocess.call()`         | Ejecuta un comando y devuelve solo el código de salida | `args`, `cwd`                                      | `ret_code = subprocess.call(["ls","-l"])`                                                                    |
| `subprocess.check_output()` | Ejecuta un comando y devuelve la salida como string    | `args`, `stderr`, `text`                           | `output = subprocess.check_output(["ls","-l"], text=True)`                                                   |

A continuación utilizaremos estas herramientas para la descarga de los archivos **SRR031714** y **SRR031715**, en un directorio específico.

In [ ]:
# Crear la ruta del directorio de entrada y salida de datos



# Hacer el prefetch de los archivos srr





En este fragmento de código definimos primero el directorio donde se guardarán los datos (`output_dir`). Luego utilizamos la función `subprocess.run()` para ejecutar el comando `prefetch` del **SRA Toolkit**, indicando el identificador de cada archivo (**SRR031714** y **SRR031715**) junto con la opción `--output-directory`, que especifica la ruta de destino. La opción `check=True` asegura que, si ocurre un error en la ejecución del comando, Python detenga el script y muestre el error.


Una vez descargados los archivos **SRR**, el siguiente paso es convertirlos al formato **FASTQ**, que es el estándar para trabajar con secuencias crudas. El siguiente código realiza la conversión para los archivos **SRR031714** y **SRR031715**, utilizando la instrucción `fastq-dump`.

In [ ]:
# Obtener las secuencias en formato fastq






En este caso las opciones empleadas en el `fastq-dump`, permiten:

- `-O`: especifica el directorio de salida donde se guardarán los archivos FASTQ.  
- `--split-files`: separa las lecturas si se trata de datos pareados (paired-end).  
- `--gzip`: comprime automáticamente los archivos FASTQ resultantes en formato `.gz`. 

Recordemos que la muestra **GSM461177** está representada por **dos archivos SRR(SRR031714 y SRR031715)**. En este caso, es necesario **concatenar** las lecturas *forward* (R1) y las lecturas *reverse* (R2) en dos archivos únicos que representen la muestra completa. El siguiente código muestra cómo realizar este proceso desde Python utilizando `subprocess`

> Nota: usamos zcat + gzip porque los archivos están comprimidos en formato .gz.

In [ ]:
# Creando lista con nombres de archivos a concatenar




# Creando los nombres de archivos de salida



# Concatenar los archivos R1
# Abrir el archivo de salida para escribir


# Ejecuta el comando zcat file1 file2, su salida (stdout) se envía a un PIPE en lugar de a la terminal.

    
# Lanza el proceso gzip. Su entrada (stdin) se conecta al stdout de p1 y su salida (stdout) se guarda directamente en el archivo fout.

    
# Se cierra el canal de salida de p1 para evitar bloqueos. Esto indica que no habrá más datos enviados a gzip.

    
# Espera a que gzip termine y asegura que toda la información se haya escrito correctamente en el archivo final.
 


# Concatenar R2



En este flujo, usamos `subprocess.Popen` junto con un **PIPE** porque necesitamos **conectar dos procesos**: primero descomprimir los archivos con `zcat`, para concatenarlos y luego recomprimirlos con `gzip`. A diferencia de `subprocess.run()`, que solo ejecuta un comando y espera a que termine, `Popen` permite manejar la **entrada y salida entre procesos en tiempo real**, lo que resulta esencial para concatenar archivos comprimidos sin generar archivos intermedios.

En este caso, el primer `subprocess.Popen` ejecuta la instrucción `zcat` y envía su salida a un **PIPE**, la cual será utilizada, como entrada, por el segundo `subprocess.Popen`, que ejecuta el comando `gzip` y cuyo resultado se guarda en el archivo `fout`.



## 5. Descarga del genoma de referencia

### 5.1 Consultando la información relacionada al genoma de referencia

Una vez descargadas las secuencias asociadas al proyecto, es necesario obtener el **genoma de referencia**, que se utilizará para mapear las lecturas de RNA-Seq y realizar posteriormente el análisis de expresión diferencial.

Para ello, utilizaremos la base de datos `assembly` de NCBI, consultando los genomas completos asociados a ***Drosophila melanogaster*** a través del módulo **Entrez** de Biopython.

In [ ]:
# Asignando el organismo
organism = "drosophila melanogaster"

# Consultando la base de datos assembly



# Imprimiendo los IDs





La consulta devuelve diversos parametros, como ya lo hemos visto en secciones anteriores. Los **UIDs de los ensamblajes** asociados al organismo consultado se encuentran en el parámetro **IdList** y se utilizarán para obtener información detallada de cada genoma en la base de datos `assembly`.

In [ ]:
# Para cada UID obtener la información
assembly_data = []

   
# Guardamos todo el diccionario
  
    


Una vez obtenida la información de cada genoma, la organizaremos en un DataFrame de pandas, lo que facilitará su manejo y exploración. En este sentido, primero podemos visualizar todas las columnas disponibles en el DataFrame y, posteriormente, seleccionar únicamente aquellas que resulten de interés.

In [ ]:
# Crear DataFrame con toda la información




# Mostrar todas las columnas disponibles




En este caso, seleccionaremos las columnas **Organism**, **AssemblyName**, **AssemblyStatus**, **AssemblyAccession**, **RefSeq_category** y **FtpPath_GenBank**. Estas nos permitirán obtener información relevante del genoma para su posterior descarga, como su **estatus** (nivel de ensamblaje), el **identificador del ensamblaje** y la **ruta de descarga** correspondiente.

In [ ]:
# Seleccionar solo algunas columnas de interés
df_selected = df_assembly[[
    "Organism",
    "AssemblyName",
    "AssemblyStatus",
    "AssemblyAccession",
    "RefSeq_category",
    "FtpPath_GenBank"
]]

# Mostrar los primeros 10 renglones del dataframe



# Filtrar filas que coincidan exactamente con el accession


# Mostrar resultado




### 5.2 Descarga de la referencia

Antes de descargar los archivos del genoma, primero exploraremos **qué información está disponible** en el directorio del ensamblaje seleccionado. Esto nos permitirá identificar los archivos de interés, como la secuencia en formato **FASTA** y la anotación en **GFF**, y asegurarnos de que existan en el servidor de NCBI.  

A continuación, se muestra cómo obtener la lista de archivos disponibles en el directorio FTP correspondiente al genoma de referencia elegido, utilizando la biblioteca `ftplib` y `urllib` de Python:

In [ ]:
# cargando las bibliotecas 
from ftplib import FTP
from urllib.parse import urlparse

# Seleccionando el path de ftp del genoma de referencia
ftp_url = df_selected.iloc[1]["FtpPath_GenBank"]

# Obteniendo la información del url
parsed = urlparse(ftp_url)

# Asignando el hostname y el path
ftp_server = parsed.hostname        # 'ftp.ncbi.nlm.nih.gov'
ftp_path = parsed.path              # '/genomes/all/GCA/038/396/815/GCA_038396815.1_Drosophila_melanogaster_ANU-1/'

print("Servidor FTP:", ftp_server)
print("Ruta en el servidor:", ftp_path)

# Haciendo la conexión al servidor de forma anónima y moviendose al path del genoma
ftp = FTP(ftp_server)
ftp.login()  # conexión anónima
ftp.cwd(ftp_path)

# Obteniendo la lista de archivos contenidos en el path
files = ftp.nlst()  # lista de archivos en el directorio

# Cerrando la conexión ftp
ftp.quit()

# Imprimiendo la lista de archivos
print("Archivos disponibles:")
for f in files:
    print(f)

Una vez que tenemos la **lista de archivos disponibles**, podemos proceder a descargar los archivos de secuencias (**FASTA**) y de anotación (**GFF**), siempre que existan en el directorio del ensamblaje seleccionado. Esto nos asegura que trabajaremos con la versión correcta del genoma y sus anotaciones asociadas, evitando errores por rutas o archivos inexistentes.

Primero, filtramos los archivos de interés:


In [ ]:
# Obtener archivos FASTA y GFF






A partir de estos resultados, descargamos los archivos usando `urllib`:

In [ ]:
import urllib.request

if fasta_files or gff_files:
    # Carpeta donde guardaremos
    out_dir = output_dir + "/genome"
    os.makedirs(out_dir, exist_ok=True)
    # Descarga de archivo FASTA
    if fasta_files:
        # Obtener el nombre base del ensamblaje (ej. GCA_000001215.4_Drosophila_melanogaster...)
        fasta_url = ftp_url + "/" + fasta_files[0]
        print("FASTA:", fasta_url)
        # Archivo de salida
        fasta_out = os.path.join(out_dir, fasta_files[0])
        # Descargar FASTA
        #urllib.request.urlretrieve(fasta_url, fasta_out)
        print("Genoma descargado en:", fasta_out)
    # Descarga de archivo GFF    
    if gff_files:
        gff_url = ftp_url + "/" + gff_files[0]
        print("GFF:", gff_url)
        # Archivo de salida
        gff_out = os.path.join(out_dir,gff_files[0])
        # 4. Descargamos el GFF
        #urllib.request.urlretrieve(gff_url, gff_out)
        print("Archivo GFF descargado en:", gff_out)


Este enfoque garantiza que solo se descarguen los archivos que efectivamente existen en el directorio del ensamblaje.

## 6. Puntos claves

En esta clase aprendimos a explorar y descargar datos de RNA-Seq y genomas de referencia usando **Biopython** y **SRA Toolkit**. Los puntos principales son:

- **Exploración de datos con Biopython:**  
  Podemos buscar proyectos (**BioProject**), estudios (**GEO / GSE**) y muestras (**GSM / SRR**) en NCBI, y ver información como título del estudio, organismo y número de muestras.

- **Jerarquía de información:**  
  La información sigue un orden:  
  `BioProject → BioSample → SRA / GEO (GSE → GSM)`  
  Esto nos ayuda a ubicar rápidamente las muestras que necesitamos.

- **Descarga de secuencias con SRA Toolkit:**  
  - `prefetch` descarga los archivos SRR completos.  
  - `fastq-dump` convierte esos archivos a **FASTQ**, listos para análisis.  
  - Si una muestra tiene más de un archivo SRR, debemos **concatenar** las lecturas *forward* (R1) y *reverse* (R2) para tener un archivo final por muestra.

- **Descarga del genoma de referencia:**  
  - Buscamos ensamblajes completos en la base `assembly` de Entrez y elegimos la versión de referencia.  
  - Obtenemos la ruta **FTP** de los archivos **FASTA** (secuencias) y **GFF** (anotaciones).  
  - Con **FTP** y `urllib` en Python, descargamos los archivos a nuestra computadora, asegurándonos de trabajar con la versión correcta del genoma.

> En resumen, aprendimos a ir **desde la búsqueda de datos en NCBI hasta tener las secuencias y genomas listos** para análisis de expresión génica, todo de manera automatizada y reproducible.

## 7. Ejercicio propuesto

Generar un *pipeline* que, dado un proyecto **PRJNAXXXX**, realice las siguientes tareas:  

1. Consultar y desplegar la información general del proyecto.  
2. Mostrar las bases de datos con las que está asociado (por ejemplo, GEO).  
3. Recuperar y presentar la información general de las muestras relacionadas.  
4. Descargar las secuencias correspondientes a cada muestra.  
   - En caso de que una muestra tenga más de un archivo de secuencias, concatenarlos, considerando si se trata de archivos *paired-end* o *single-read*.  
5. Consultar y mostrar la información del genoma de referencia, si está disponible.  

---

### Requisitos del pipeline  

- Debe ser **modular**, con funciones independientes para cada tarea.  
- Debe incluir **validaciones** en cada etapa:  
  - Verificar accesos válidos al BioProject y a las bases de datos asociadas.  
  - Confirmar la existencia de archivos antes de descargarlos o concatenarlos.  
  - Comprobar que los formatos sean correctos (FASTQ, referencia en FASTA, etc.).  
- Las tareas deben **optimizarse** mediante:  
  - **Ciclos** para procesar múltiples muestras de manera eficiente.  
  - **Decisiones condicionales** para diferenciar entre archivos *paired-end* y *single-read*.  
  - **Uso de scripts externos en Bash** cuando sea más eficiente que manejar procesos con `subprocess.PIPE`.  
    - Ejemplo: descarga masiva con `prefetch` o `fasterq-dump`, o concatenación de archivos con `cat`.  

---

**Nota:** La estructura debe facilitar la reutilización en distintos proyectos, garantizando claridad, escalabilidad y eficiencia.  

# Referencias

+ Brooks AN, Yang L, Duff MO, Hansen KD, Park JW, Dudoit S, Brenner SE, Graveley BR. Conservation of an RNA regulatory map between Drosophila and mammals. Genome Res. 2011 Feb;21(2):193-202. doi: 10.1101/gr.108662.110. Epub 2010 Oct 4. PMID: 20921232; PMCID: PMC3032923.